Fine-tuning with transformers
===

Reminding myself about some of the details of finetuning.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import transformers